# Demo for GenPRM inference

## Import inference package

In [1]:
import os
import sys
sys.path.append('..')
from prm_evaluation.genprm_inference import GenPRM

/mnt/weka/aisg/ob1/GenPRM/src/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-05-20 04:07:18,755	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


INFO 05-20 04:07:21 __init__.py:183] Automatically detected platform cuda.


## initialize GenPRM with vllm

In [6]:
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

genprm = GenPRM('GenPRM/GenPRM-7B', tensor_parallel_size=1)

[2025-05-20 04:13:47] [INFO] Loading model from GenPRM/GenPRM-7B
INFO 05-20 04:13:51 config.py:526] This model supports multiple tasks: {'reward', 'generate', 'classify', 'embed', 'score'}. Defaulting to 'generate'.
INFO 05-20 04:13:51 config.py:1538] Chunked prefill is enabled with max_num_batched_tokens=2048.
INFO 05-20 04:13:51 llm_engine.py:232] Initializing a V0 LLM engine (v0.7.1) with config: model='GenPRM/GenPRM-7B', speculative_config=None, tokenizer='GenPRM/GenPRM-7B', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=131072, download_dir=None, load_format=auto, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='xgrammar'), observability_config=ObservabilityConfig(otlp_traces_endpoint=

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  25% Completed | 1/4 [00:01<00:04,  1.56s/it]
Loading safetensors checkpoint shards:  50% Completed | 2/4 [00:02<00:02,  1.47s/it]
Loading safetensors checkpoint shards:  75% Completed | 3/4 [00:03<00:00,  1.01it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:04<00:00,  1.20s/it]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:04<00:00,  1.23s/it]



INFO 05-20 04:17:30 model_runner.py:1116] Loading model weights took 14.2717 GB
INFO 05-20 04:17:31 worker.py:266] Memory profiling takes 0.62 seconds
INFO 05-20 04:17:31 worker.py:266] the current vLLM instance can use total_gpu_memory (139.72GiB) x gpu_memory_utilization (0.90) = 125.75GiB
INFO 05-20 04:17:31 worker.py:266] model weights take 14.27GiB; non_torch_memory takes 0.15GiB; PyTorch activation peak memory takes 1.43GiB; the rest of the memory reserved for KV Cache is 109.90GiB.
INFO 05-20 04:17:31 executor_base.py:108] # CUDA blocks: 128611, # CPU blocks: 4681
INFO 05-20 04:17:31 executor_base.py:113] Maximum concurrency for 131072 tokens per request: 15.70x
INFO 05-20 04:17:33 model_runner.py:1435] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error occurs during cudagraph capture, consider decreasing `gpu_m

Capturing CUDA graph shapes: 100%|███████████████████████████████████████████████████████████████████████| 35/35 [00:12<00:00,  2.76it/s]


INFO 05-20 04:17:46 model_runner.py:1563] Graph capturing finished in 13 secs, took 0.31 GiB
INFO 05-20 04:17:46 llm_engine.py:429] init engine (profile, create kv cache, warmup model) took 15.70 seconds
[2025-05-20 04:17:48] [INFO] GenPRM loaded successfully


## Pass@1 inference for single step

In [7]:
from prm_evaluation.genprm_inference import CodeExecutor
from utils.util import *

messages = [ 
    { "content": "You are a math teacher. Your task is to review and critique the paragraphs in solution step by step.", "role": "system" }, 
    { "content": "Question: Jo adds up all the positive integers from 1 to 100. Kate does a similar thing with the first 100 positive integers; however, she first rounds every integer to its nearest multiple of 10 (rounding 5s up) and then adds the 100 values. What is the positive difference between Jo's sum and Kate's sum?\n\nFirst, we need to calculate Jo's sum, which is the sum of all positive integers from 1 to 100. This can be directly computed using the formula for the sum of the first \\(n\\) positive integers, which is \\(\\frac{n(n+1)}{2}\\). For \\(n = 100\\), Jo's sum is \\(\\frac{100 \\cdot 101}{2} = 5050\\).", "role": "user" }, 
]
code_executor = CodeExecutor()
output, reward = genprm.inference(messages, cur_step=1, code_executor=code_executor)

cprint(output, 'Model Output')
cprint(reward, 'Reward')

****************************************
Start: paragraph 1 request 1
----------------------------------------
<｜begin▁of▁sentence｜>You are a math teacher. Your task is to review and critique the paragraphs in solution step by step.<｜User｜>Question: Jo adds up all the positive integers from 1 to 100. Kate does a similar thing with the first 100 positive integers; however, she first rounds every integer to its nearest multiple of 10 (rounding 5s up) and then adds the 100 values. What is the positive difference between Jo's sum and Kate's sum?\n\nFirst, we need to calculate Jo's sum, which is the sum of all positive integers from 1 to 100. This can be directly computed using the formula for the sum of the first \(n\) positive integers, which is \(\frac{n(n+1)}{2}\). For \(n = 100\), Jo's sum is \(\frac{100 \cdot 101}{2} = 5050\).<analyze>\nLet's analyze the Paragraph 1 step by step: 
----------------------------------------
End: paragraph 1 request 1
*************************************

## Majority of 8 inference for single step

In [8]:
from prm_evaluation.genprm_inference import CodeExecutor
from utils.util import *

messages = [ 
    { "content": "You are a math teacher. Your task is to review and critique the paragraphs in solution step by step.", "role": "system" }, 
    { "content": "Question: Jo adds up all the positive integers from 1 to 100. Kate does a similar thing with the first 100 positive integers; however, she first rounds every integer to its nearest multiple of 10 (rounding 5s up) and then adds the 100 values. What is the positive difference between Jo's sum and Kate's sum?\n\nFirst, we need to calculate Jo's sum, which is the sum of all positive integers from 1 to 100. This can be directly computed using the formula for the sum of the first \\(n\\) positive integers, which is \\(\\frac{n(n+1)}{2}\\). For \\(n = 100\\), Jo's sum is \\(\\frac{100 \\cdot 101}{2} = 5050\\).", "role": "user" }, 
]
code_executor = CodeExecutor()
output, reward = genprm.inference(messages, majority_num=8, cur_step=1, code_executor=code_executor)

# cprint(output, 'Model Output')
cprint(reward, 'Reward')

****************************************
Start: paragraph 1 request 1
----------------------------------------
<｜begin▁of▁sentence｜>You are a math teacher. Your task is to review and critique the paragraphs in solution step by step.<｜User｜>Question: Jo adds up all the positive integers from 1 to 100. Kate does a similar thing with the first 100 positive integers; however, she first rounds every integer to its nearest multiple of 10 (rounding 5s up) and then adds the 100 values. What is the positive difference between Jo's sum and Kate's sum?\n\nFirst, we need to calculate Jo's sum, which is the sum of all positive integers from 1 to 100. This can be directly computed using the formula for the sum of the first \(n\) positive integers, which is \(\frac{n(n+1)}{2}\). For \(n = 100\), Jo's sum is \(\frac{100 \cdot 101}{2} = 5050\).<analyze>\nLet's analyze the Paragraph 1 step by step: 
----------------------------------------
End: paragraph 1 request 1
*************************************

## process supervision for whole solution

In [9]:
messages = [ 
	{ "content": "You are a math teacher. Your task is to review and critique the paragraphs in solution step by step.", "role": "system" }, 
	{ "role": "user", "content": "Question: What numeral is in the 100th decimal place in the decimal representation of $\\frac{6}{7}$?\n\nTo find the numeral in the 100th decimal place of $\\frac{6}{7}$, we first need to determine the decimal representation of $\\frac{6}{7}$." }, 
	{ "role": 'assistant', 'content': ''},
	{ "role": "user", "content": "Performing the division, we find that $\\frac{6}{7} = 0.\\overline{857142}$, which means the decimal representation is a repeating decimal with a period of 6." }, 
	{ "role": 'assistant', 'content': ''},
	{ "role": "user", "content": "The repeating block is \"857142\", and it repeats indefinitely." }, 
	{ "role": 'assistant', 'content': ''},
	{ "role": "user", "content": "To find the numeral in the 100th decimal place, we need to determine the position of the 100th decimal within this repeating block." }, 
	{ "role": 'assistant', 'content': ''},
	{ "role": "user", "content": "We do this by finding the remainder when 100 is divided by the length of the repeating block, which is 6." }, 
	{ "role": 'assistant', 'content': ''},
	{ "role": "user", "content": "Calculating the remainder, we get $100 \\div 6 = 16$ with a remainder of 4. This means the 100th decimal place corresponds to the 4th digit in the repeating block \"857142\"." }, 
	{ "role": 'assistant', 'content': ''},
	{ "role": "user", "content": "Therefore, the numeral in the 100th decimal place is the 4th digit of the repeating block, which is $4$." }, 
	{ "role": 'assistant', 'content': ''},
	{ "role": "user", "content": "The remainder of 4 indicates that the 100th decimal place is the same as the 4th digit in the repeating block, which is 4.\n\nAnswer: \\boxed{4}" },
	{ "role": 'assistant', 'content': ''},
]

code_executor = CodeExecutor()
reward_list = []
for i in range(len(messages)):
    if messages[i]['role'] != 'assistant':
        continue
    output, reward = genprm.inference(messages[:i], cur_step=int(i/2), code_executor=code_executor, logging=False)
    messages[i]['content'] = output[0]
    reward_list.append(reward)

print(reward_list)

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0]
